##Emotion Classification

Import Libraries

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import spacy

import warnings
warnings.filterwarnings('ignore')

Load an emotion-labeled dataset

In [ ]:
df = pd.read_csv('Emotion_classify_Data.csv')

In [ ]:
print(df.shape)

(5937, 2)


In [ ]:
df.head(5)

,Comment,Emotion
0,i seriously hate one subject to death but now ...,fear
1,im so full of life i feel appalled,anger
2,i sit here to write i start to dig out my feel...,fear
3,ive been really angry with r and i feel like a...,joy
4,i feel suspicious if there is no one outside l...,fear


In [ ]:
df['Emotion'].value_counts()

,count
Emotion,
anger,2000
joy,2000
fear,1937


## Preprocess

In [ ]:
nlp = spacy.load('en_core_web_sm')#ingilis modeli

In [ ]:
txt = df['Comment'][3]
doc = nlp(txt)
doc

ive been really angry with r and i feel like an idiot for trusting him in the first place

In [ ]:
for token in doc:
    print(f"Word: {token} | -> {token.lemma_}")

Word: i | -> I
Word: ve | -> ve
Word: been | -> be
Word: really | -> really
Word: angry | -> angry
Word: with | -> with
Word: r | -> r
Word: and | -> and
Word: i | -> I
Word: feel | -> feel
Word: like | -> like
Word: an | -> an
Word: idiot | -> idiot
Word: for | -> for
Word: trusting | -> trust
Word: him | -> he
Word: in | -> in
Word: the | -> the
Word: first | -> first
Word: place | -> place


In [ ]:
for token in doc:
    if token.is_stop or token.is_punct:
        print(token)

i
been
really
with
and
i
an
for
him
in
the
first


In [ ]:
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)

    return " ".join(filtered_tokens)
procces_txt = preprocess(txt)

In [ ]:
df['preprocessed_comment'] = df['Comment'].apply(preprocess)

## Encoding

In [ ]:
df['Emotion_num'] = df['Emotion'].map({'joy' : 0, 'fear': 1, 'anger': 2})

df.head(5)

,Comment,Emotion,preprocessed_comment,Emotion_num
0,i seriously hate one subject to death but now ...,fear,seriously hate subject death feel reluctant drop,1
1,im so full of life i feel appalled,anger,m life feel appalled,2
2,i sit here to write i start to dig out my feel...,fear,sit write start dig feeling think afraid accep...,1
3,ive been really angry with r and i feel like a...,joy,ve angry r feel like idiot trust place,0
4,i feel suspicious if there is no one outside l...,fear,feel suspicious outside like rapture happen,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['preprocessed_comment'], df['Emotion_num'],
                                                    test_size=.2, random_state=123, stratify=df['Emotion_num'])
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (4749,)
Shape of X_test:  (1188,)


In [ ]:
v = TfidfVectorizer()

X_train_cv = v.fit_transform(X_train)
X_test_cv = v.transform(X_test)

print(v.vocabulary_)

{'feel': 1958, 'reluctant': 4445, 'talk': 5341, 'death': 1283, 'happen': 2400, 'school': 4673, 'initially': 2743, 'want': 5907, 'remember': 4452, 'specific': 5033, 'thing': 5450, 'class': 884, 'feeling': 1961, 'terrify': 5418, 'teacher': 5372, 'repeat': 4467, 'question': 4281, 'spanish': 5023, 'increase': 2705, 'volume': 5881, 'victim': 5845, 'manage': 3265, 'answer': 231, 'correctly': 1122, 'run': 4588, 'away': 381, 'scream': 4687, 'distract': 1505, 'cute': 1232, 'boy': 612, 'help': 2479, 'study': 5193, 'test': 5424, 'seriously': 4754, 'hate': 2427, 'subject': 5210, 'drop': 1582, 'appreciative': 271, 'get': 2239, 'home': 2530, 'start': 5109, 'weird': 5959, 'woman': 6036, 'wear': 5940, 'boot': 591, 'envious': 1760, 'curse': 1227, 'kind': 2988, 'left': 3082, 'insecure': 2759, 'uneasy': 5697, 'skin': 4886, 'know': 3007, 'step': 5130, 'candyland': 727, 'look': 3183, 'haggard': 2375, 'leave': 3080, 'radiant': 4301, 'bag': 403, 'product': 4190, 'course': 1139, 'eager': 1610, 'come': 962, 'd

#Random forest

In [ ]:
RFC_model = RandomForestClassifier()

RFC_model.fit(X_train_cv, y_train)

RandomForestClassifier()

In [ ]:
y_pred = RFC_model.predict(X_test_cv)

In [ ]:
print(accuracy_score(y_test, y_pred))

0.9250841750841751


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94       400
           1       0.91      0.94      0.92       388
           2       0.93      0.91      0.92       400

    accuracy                           0.93      1188
   macro avg       0.93      0.93      0.93      1188
weighted avg       0.93      0.93      0.93      1188



## Test

In [ ]:
test_text = df['Comment'][2000]
test_text

'im looking good and feeling good other than this crappy cold im dealing with'

In [ ]:
test_text_processed  = [preprocess(test_text)]
test_text_processed

['m look good feel good crappy cold m deal']

In [ ]:
test_text_vc = v.transform(test_text_processed)#vectora cevir

In [ ]:
test_text = RFC_model.predict(test_text_vc)

In [ ]:
print(f"{df['Emotion'][2000]} -> {df['Emotion_num'][2000]}")
print(test_text)

joy -> 0
[0]
